In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf 
import os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras_preprocessing.image import load_img
# from tensorflow.keras.preprocessing.image import load_img

from tqdm import tqdm
from tensorflow.keras.utils import to_categorical



In [13]:
TrainDir = r"images/train" 
TestDir = r"images/validation"

In [14]:
def createDataFrame(Dir):
  imgPaths = []
  imgLabels = []
  
  for label in os.listdir(Dir):
    for img in os.listdir(os.path.join(Dir,label)):
      imgPaths.append(os.path.join(Dir,label,img))
      imgLabels.append(label)
    print(label, "Completed")
  return imgPaths, imgLabels

In [15]:
train = pd.DataFrame()
test = pd.DataFrame()

train['images'], train['labels'] = createDataFrame(TrainDir)
print()
test['images'], test['labels'] = createDataFrame(TestDir)

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed

angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed
surprise Completed


In [16]:
train

,images,labels
0,images/train\angry\0.jpg,angry
1,images/train\angry\1.jpg,angry
2,images/train\angry\10.jpg,angry
3,images/train\angry\10002.jpg,angry
4,images/train\angry\10016.jpg,angry
...,...,...
28816,images/train\surprise\9969.jpg,surprise
28817,images/train\surprise\9985.jpg,surprise
28818,images/train\surprise\9990.jpg,surprise
28819,images/train\surprise\9992.jpg,surprise


In [17]:
test

,images,labels
0,images/validation\angry\10052.jpg,angry
1,images/validation\angry\10065.jpg,angry
2,images/validation\angry\10079.jpg,angry
3,images/validation\angry\10095.jpg,angry
4,images/validation\angry\10121.jpg,angry
...,...,...
7061,images/validation\surprise\9806.jpg,surprise
7062,images/validation\surprise\9830.jpg,surprise
7063,images/validation\surprise\9853.jpg,surprise
7064,images/validation\surprise\9878.jpg,surprise


In [18]:
def featureExtraction(images):
    imgFeatures = []
    
    for image in tqdm(images):
        img = load_img(image,color_mode='grayscale')
        img = np.array(img)
        imgFeatures.append(img)
    imgFeatures = np.array(imgFeatures)
    imgFeatures = imgFeatures.reshape(len(imgFeatures),48,48,1)
    
    return imgFeatures


        

In [19]:
train_features = featureExtraction(train['images'])

100%|██████████| 28821/28821 [01:23<00:00, 347.22it/s]


In [20]:
test_features = featureExtraction(test['images'])

100%|██████████| 7066/7066 [00:31<00:00, 224.31it/s]


In [21]:
x_train = train_features/255.0
x_test = test_features/255.0


In [22]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
le.fit(train['labels'])

y_train = le.transform(train['labels'])
y_test = le.transform(test['labels'])

In [23]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)

In [24]:
# model = Sequential()

# model.add(Conv2D(128,kernel_size=(3,3), activation = 'relu', input_shape=(48,48,1)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(256,kernel_size=(3,3), activation = 'relu', input_shape=(48,48,1)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512,kernel_size=(3,3), activation = 'relu', input_shape=(48,48,1)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512,kernel_size=(3,3), activation = 'relu', input_shape=(48,48,1)))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Flatten())


# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.4))

# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.3))

# model.add(Dense(7,activation = 'softmax'))


# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')


In [25]:
# model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test,y_test))



# model_json = model.to_json()
# with open("emothionDetector.json", 'w') as json_file:
#     json_file.write(model_json)
# model.save("emothionDetector.h5")

In [26]:
from keras.models import model_from_json

In [72]:
json_file = open("emothionDetector.json", 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emothionDetector.h5")


label = ['angry','disgust','fear','happy','neutral','sad','surprise']

def ef(image):
    img = load_img(image,color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0


image = 'images/train/fear/49.jpg'
print("Original Image is of fear")


img = ef(image)

pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model predictoin is : ", pred_label)



# plt.imshow(img.reshape(48,48), cmap='gray')



Original Image is of fear
1/1 [==============================] - 0s 56ms/step
model predictoin is :  fear
